## PartD: prescriber level opioid dosage sum-> MME >120 indicator  => aggregate to census track level to indicate potential opioid overdose severity

In [1]:
#importing various packages I expect to use
import json
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt 
import scipy 
import scipy.stats as stats
import math
from scipy import stats
import pandas as pd
import numpy as np
import requests
import time
import json

## Loading all the data into Pandas DataFrames

In [2]:
#data = pandas.read_sas('PartD_Prescriber_PUF_NPI_14-SAS-Infile.sas')
PUFNPI = pd.read_csv('PartD_Prescriber_PUF_NPI_14.txt',sep='\t')
PUFNPI = pd.DataFrame(PUFNPI)
CincinnatiBuprPhysLoc = pd.read_csv('Physician_Locator_2018-04-30T17-01-58.csv')
CincinnatiBuprPhysLoc = pd.DataFrame(CincinnatiBuprPhysLoc)
BupronephrineTreatmentFacility  = pd.read_excel('Behavioral_Health_Treament_Facility_listing_2018_05_01_021054.xlsx',sheetname='Facilities with service detail')
BupronephrineTreatmentFacilityREF  = pd.read_excel('Behavioral_Health_Treament_Facility_listing_2018_05_01_021054.xlsx',sheetname='service code reference')

## The Part D Prescriber PUF is organized by National Provider Identifier (NPI) and drug name and contains information on drug utilization (claim counts and day supply) and total drug costs. The data in the Part D Prescriber PUF cover calendar years 2013 through 2015.
### PUFNPIDrug: The PUF NPI Drug (24M rows x 19 columns, rows indexed at NPI+Drug level) file has drug dispensing information and pattern at the NPI level. It has general metrics such as distinct number of  patients on that drug, drugs prescribes , cost of those prescriptions. Bene count is how many patients are on a specific drug. (This table might need to be matched to another drug table to get drug class information, etc.)
### PUFNPI: While the PUF NPI  table (1M rows x 109 columns) has information regarding the provider. It has aggregate Rx info like total drug cost/days supply/30-day fills, and additionally, address/zip-code, generic vs. brand name Rx meta data. It also includes aggreate data on overal Opioid class. 
  #### This information includes opioid_claim_count, opioid_drug_cost, opioid_day_supply, opioid_bene_count, opioid_prescriber_rate. other info include lis_claim_count (Low Income Subsidized drugs), anti-psych meds info,  race-count of patients for that NPI, and average HCC risk.
    ### CincinnatiBuprPhysLoc: 
## Ended up only using the PUFNPI and Bupronephrine Treatment centers' data. So saving room by not importing othere datasets  


In [3]:
CinciPUFNPI = PUFNPI[(PUFNPI['nppes_provider_city']=='CINCINNATI')&(PUFNPI['nppes_provider_state']=='OH')]
CinciBupTrtmntFac = BupronephrineTreatmentFacility[(BupronephrineTreatmentFacility['city']=='Cincinnati')&(BupronephrineTreatmentFacility['state']=='OH')]

In [4]:
print('CinciPUFNPI :',type(CinciPUFNPI), CinciPUFNPI.shape)
print('CincinnatiBuprPhysLoc :',type(CincinnatiBuprPhysLoc), CincinnatiBuprPhysLoc.shape)
print('CinciBupTrtmntFac :',type(CinciBupTrtmntFac), CinciBupTrtmntFac.shape)

CinciPUFNPI : <class 'pandas.core.frame.DataFrame'> (4363, 84)
CincinnatiBuprPhysLoc : <class 'pandas.core.frame.DataFrame'> (175, 11)
CinciBupTrtmntFac : <class 'pandas.core.frame.DataFrame'> (32, 209)


In [5]:
print('CinciPUFNPI')
print(CinciPUFNPI.info())
print(CinciPUFNPI.columns)
print('------------------------------------------------------------------')
print('CincinnatiBuprPhysLoc')
print(CincinnatiBuprPhysLoc.info())
print(CincinnatiBuprPhysLoc.columns)
print('------------------------------------------------------------------')
print('CinciBupTrtmntFac')
print(CinciBupTrtmntFac.info())
print(CinciBupTrtmntFac.columns)

CinciPUFNPI
<class 'pandas.core.frame.DataFrame'>
Int64Index: 4363 entries, 769 to 1072666
Data columns (total 84 columns):
npi                                 4363 non-null int64
nppes_provider_last_org_name        4363 non-null object
nppes_provider_first_name           4363 non-null object
nppes_provider_mi                   3134 non-null object
nppes_credentials                   4127 non-null object
nppes_provider_gender               4363 non-null object
nppes_entity_code                   4363 non-null object
nppes_provider_street1              4363 non-null object
nppes_provider_street2              2067 non-null object
nppes_provider_city                 4363 non-null object
nppes_provider_zip5                 4363 non-null float64
nppes_provider_zip4                 3982 non-null float64
nppes_provider_state                4363 non-null object
nppes_provider_country              4363 non-null object
specialty_description               4363 non-null object
description_flag    

In [6]:
print('CinciPUFNPI :', PUFNPI.head())
print('CincinnatiBuprPhysLoc :', CincinnatiBuprPhysLoc.head())
print('CinciBupTrtmntFac :', BupronephrineTreatmentFacility.head())

CinciPUFNPI :           npi nppes_provider_last_org_name nppes_provider_first_name  \
0  1518048750                      A VILES                      OMAR   
1  1952310666                    A'BODJEDI                    ENENGE   
2  1467482471                          AAB                     BARRY   
3  1801093968                       AABERG                   MELANIE   
4  1588763981                       AABERG                    RANDAL   

  nppes_provider_mi nppes_credentials nppes_provider_gender nppes_entity_code  \
0                 P                MD                     M                 I   
1               NaN                MD                     M                 I   
2                 N            D.D.S.                     M                 I   
3                 L                DO                     F                 I   
4                 A                MD                     M                 I   

  nppes_provider_street1            nppes_provider_street2  \
0   

### Limiting the datasets to only data from Cincinnati, OH

In [7]:
CinciPUFNPI = CinciPUFNPI.drop(['description_flag','medicare_prvdr_enroll_status','mapd_suppress_flag','mapd_claim_count','mapd_drug_cost','pdp_suppress_flag','pdp_claim_count','pdp_drug_cost','lis_suppress_flag','lis_claim_count','lis_drug_cost','nonlis_suppress_flag','nonlis_claim_count','nonlis_drug_cost','antibiotic_claim_count','antibiotic_drug_cost','antibiotic_bene_count','hrm_ge65_suppress_flag','hrm_claim_count_ge65','hrm_drug_cost_ge65','hrm_bene_ge65_suppress_flag','hrm_bene_count_ge65'],axis=1)

In [8]:
CinciPUFNPI.head()

,npi,nppes_provider_last_org_name,nppes_provider_first_name,nppes_provider_mi,nppes_credentials,nppes_provider_gender,nppes_entity_code,nppes_provider_street1,nppes_provider_street2,nppes_provider_city,...,beneficiary_male_count,beneficiary_race_white_count,beneficiary_race_black_count,beneficiary_race_asian_pi_count,beneficiary_race_hispanic_count,beneficiary_race_nat_ind_count,beneficiary_race_other_count,beneficiary_nondual_count,beneficiary_dual_count,beneficiary_average_risk_score
769,1932191681,ABBOTT,CAROL,J,CNP,F,I,8444 WINTON RD,NaN,CINCINNATI,...,14.0,NaN,NaN,0.0,0.0,0.0,0.0,NaN,NaN,0.8015
783,1750412615,ABBOTT,DANIEL,E,MD,M,I,222 PIEDMONT AVE.,NaN,CINCINNATI,...,13.0,NaN,NaN,0.0,0.0,0.0,0.0,NaN,NaN,2.2548
913,1568414290,ABBOTT,SUSAN,B,MD,F,I,4600 MCAULEY PLACE,SUITE 115,CINCINNATI,...,22.0,104.0,NaN,NaN,0.0,0.0,NaN,105.0,21.0,1.1052
934,1346226388,ABBOTTSMITH,CHARLES,W,M.D.,M,I,2123 AUBURN AVE,SUITE 139,CINCINNATI,...,322.0,487.0,35.0,NaN,0.0,0.0,NaN,479.0,56.0,1.3869
1090,1972714251,ABDALLAH,MOUHAMAD,NaN,"MD, PHD",M,I,234 GOODMAN ST,NaN,CINCINNATI,...,49.0,83.0,NaN,NaN,NaN,0.0,NaN,66.0,40.0,2.4137


In [9]:
CinciPUFNPI['Address']=CinciPUFNPI['nppes_provider_street1']+','+CinciPUFNPI['nppes_provider_city']+','+CinciPUFNPI['nppes_provider_state']+' '+CinciPUFNPI['nppes_provider_zip5'].astype(int).astype(str)

In [10]:
CinciPUFNPI['Address'].head()

769         8444 WINTON RD,CINCINNATI,OH 45231
783      222 PIEDMONT AVE.,CINCINNATI,OH 45219
913     4600 MCAULEY PLACE,CINCINNATI,OH 45242
934        2123 AUBURN AVE,CINCINNATI,OH 45219
1090        234 GOODMAN ST,CINCINNATI,OH 45219
Name: Address, dtype: object

In [11]:
CincinnatiBuprPhysLoc.head()

,Unnamed: 0,First,Last,Deg.,Address,City,County,State,Postal Code,Tele.,Fax
0,NaN,Julie,Adams,DO,4130 Dry Ridge Road,Cincinnati,-,Ohio,45252,513-981-5162,513-923-5522
1,NaN,Anne,Adams,MD,2136 W 8th Street,Cincinnati,HAMILTON,Ohio,45204,513-357-2700,513-357-2750
2,Dr.,Celedor,Akintunde,MD,WinMed Healthcare Center - City West II1019 Li...,Cincinnati,HAMILTON,Ohio,45203,513-233-7100,513-407-3451
3,Dr.,Zaheer,Alam,MD,47 E. Hollister StreetSuite 102,Cincinnati,HAMILTON,Ohio,45219,513-276-4083,NaN
4,NaN,Alan,Altman,MD,440 Ray Norrish Drive,Cincinnati,HAMILTON,Ohio,45246,513-671-7700,513-671-5435


In [12]:
CincinnatiBuprPhysLoc['Address'] = CincinnatiBuprPhysLoc['Address']+','+CincinnatiBuprPhysLoc['City']+','+CincinnatiBuprPhysLoc['State']+' '+CincinnatiBuprPhysLoc['Postal Code'].astype(str)

In [13]:
CinciBupTrtmntFac['Address'] = CinciBupTrtmntFac['street1']+','+CinciBupTrtmntFac['city']+','+CinciBupTrtmntFac['state']+' '+CinciBupTrtmntFac['zip'].astype(int).astype(str)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [14]:
CinciBupTrtmntFac

,name1,name2,street1,street2,city,state,zip,zip4,county,phone,...,f67,f70,f81,f92,n13,n18,n23,n24,n40,Address
0,Central Clinic,Court Clinic Division,909 Sycamore Street,NaN,Cincinnati,OH,45202,NaN,Hamilton,513-352-1342,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"909 Sycamore Street,Cincinnati,OH 45202"
1,Joseph House Inc,NaN,1526 Republic Street,NaN,Cincinnati,OH,45202,NaN,Hamilton,513-241-2965,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"1526 Republic Street,Cincinnati,OH 45202"
2,Shelterhouse Volunteer Group,NaN,411 Gest Street,Suite 1,Cincinnati,OH,45203,NaN,Hamilton,513-721-0643,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"411 Gest Street,Cincinnati,OH 45203"
3,Center for Addiction Treatment,NaN,830 Ezzard Charles Drive,NaN,Cincinnati,OH,45214,NaN,Hamilton,513-381-6672,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"830 Ezzard Charles Drive,Cincinnati,OH 45214"
10,IKRON Corporation,NaN,2347 Vine Street,NaN,Cincinnati,OH,45219,NaN,Hamilton,513-621-1117,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"2347 Vine Street,Cincinnati,OH 45219"
11,Health Resource Ct of Cincinnati Inc,NaN,2347 Vine Street,NaN,Cincinnati,OH,45219,NaN,Hamilton,513-357-4602 x3070,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"2347 Vine Street,Cincinnati,OH 45219"
12,First Step Home Inc,Residential,2203 Fulton Avenue,NaN,Cincinnati,OH,45206,NaN,Hamilton,513-961-4663,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"2203 Fulton Avenue,Cincinnati,OH 45206"
14,Lighthouse Youth Services,NaN,401 East McMillan Street,NaN,Cincinnati,OH,45206,NaN,Hamilton,513-221-3350,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"401 East McMillan Street,Cincinnati,OH 45206"
15,Bethesda Alcohol and Drug Trt Prog,NaN,619 Oak Street,NaN,Cincinnati,OH,45206,NaN,Hamilton,513-569-6116,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"619 Oak Street,Cincinnati,OH 45206"
17,Addiction Services Council,NaN,2828 Vernon Place,NaN,Cincinnati,OH,45219,NaN,Hamilton,513-281-7880,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"2828 Vernon Place,Cincinnati,OH 45219"


In [15]:
"""Iris Sheu's code - Evolent Health Analyst"""

def address_to_censusblock(address_string):
    """Converts address to 12 digit census block group ID using 
    census geocoder API and returns list of ID and coordinates dictionary"""

    endpoint_url = "https://geocoding.geo.census.gov/geocoder/geographies/onelineaddress"
    census_block = -1
    coordinates = {"x": -1000., "y": -1000.}
    try:
        parameters = {
            "address": address_string,
            "format": "json",
            "searchtype": "onelineaddress",
            "benchmark":9,
            "vintage": "Census2010_Census2010",
            "layer": 14
        }
        response = requests.get(endpoint_url, params=parameters)
        data = response.json()
        census_block = data["result"]["addressMatches"][0]["geographies"]["Census Blocks"][0]["GEOID"][:12].encode("ascii")
        coordinates = {
            "x": data["result"]["addressMatches"][0]["geographies"]["Census Blocks"][0]["CENTLAT"].encode("ascii"), 
            "y": data["result"]["addressMatches"][0]["geographies"]["Census Blocks"][0]["CENTLON"].encode("ascii")
        }
    except:
        print (response.status_code)
        
    return [census_block, coordinates]

In [16]:
# Single address example
parameters = {
    "address":'17 WILLIAM H. TAFT ROAD, CINCINNATI, OH 45219',#"21 Marina Rd., Chelmsford, MA 01824",#"4600 Silver Hill Rd, Suitland, MD 20746",
    "format": "json",
    "searchtype": "onelineaddress",
    "benchmark":9,
    "vintage": "Census2010_Census2010",
    "layer": 14
}
#response = requests.get("https://geo.fcc.gov/api/census/block/find", params=parameters)
#response.content
response = requests.get("https://geocoding.geo.census.gov/geocoder/geographies/onelineaddress", params=parameters)
data=response.json()
data
#data["result"]["addressMatches"][0]["geographies"]["Census Blocks"][0]["GEOID"][:12]
#data["result"]["addressMatches"][0]["geographies"]["Census Blocks"][0]["CENTLAT"].encode("ascii"), data["result"]["addressMatches"][0]["geographies"]["Census Blocks"][0]["CENTLON"].encode("ascii")

{'result': {'addressMatches': [],
  'input': {'address': {'address': '17 WILLIAM H. TAFT ROAD, CINCINNATI, OH 45219'},
   'benchmark': {'benchmarkDescription': 'Public Address Ranges - Census 2010 Benchmark',
    'benchmarkName': 'Public_AR_Census2010',
    'id': '9',
    'isDefault': False},
   'vintage': {'id': '910',
    'isDefault': True,
    'vintageDescription': 'Census2010 Vintage - Census2010 Benchmark',
    'vintageName': 'Census2010_Census2010'}}}}

In [19]:
address_to_censusblock("8444 WINTON RD,CINCINNATI,OH 45231")

[b'390610221022', {'x': b'+39.2224212', 'y': b'-084.5156468'}]

In [25]:
start = time.time()
block_list = []
x_list = []
y_list = []
idx = 0
for address in CinciPUFNPI.Address:
    [census_block, coords_dict] = address_to_censusblock(address)
    block_list.append(census_block)
    x_list.append(coords_dict["x"])
    y_list.append(coords_dict["y"])
    idx += 1
    if idx%500 == 0:
        print (idx, "coordinates converted")
    if census_block == -1:
        print (address)
print ("total time:",(time.time() - start)/60., "mins")


200
4600 MCAULEY PLACE,CINCINNATI,OH 45242
200
231 ALBERT SABIN WAY,CINCINNATI,OH 45267
200
8251 PINE RD,CINCINNATI,OH 45236
200
3301 MERCY HEALTH BLVD,CINCINNATI,OH 45211
200
8099 CORNELL RD,CINCINNATI,OH 45249
200
231 ALBERT SABIN WAY,CINCINNATI,OH 45267
200
231 ALBERT SABIN WAY,CINCINNATI,OH 45267
200
231 ALBERT SABIN WAY,CINCINNATI,OH 45267
200
8251 PINE RD STE 212,CINCINNATI,OH 45236
200
DEPARTMENT OF ENDOCRINOLOGY 260 STETSON ST,CINCINNATI,OH 45267
200
UNIVERSITY OF CINCINNATI MEDICAL CENTER DEPT,CINCINNATI,OH 45267
200
200 ALBERT SABIN WAY,CINCINNATI,OH 45267
200
231 ALBERT SABIN WAY,CINCINNATI,OH 45267
200
UNIVERSITY OF CINCINNATI DEPARTMENT OF,CINCINNATI,OH 45267
200
500 E BUSINESS WAY,CINCINNATI,OH 45241
200
231 ALBERT SABIN WAY,CINCINNATI,OH 45267
200
231 ALBERT SABIN WAY,CINCINNATI,OH 45267
200
8251 PINE RD,CINCINNATI,OH 45236
200
231 ALBERT SABIN WAY,CINCINNATI,OH 45267
200
231 ALBERT SABIN WAY,CINCINNATI,OH 45267
200
7447 WOOSTER PIKE,CINCINNATI,OH 45227
200
231 ALBERT SA

UnboundLocalError: local variable 'response' referenced before assignment

In [26]:
tract_list = [str(block)[:11] for block in block_list]
CinciPUFNPI["CENSUS_BLOCK_ID"] = block_list
CinciPUFNPI["CENSUS_TRACT_ID"] = tract_list
CinciPUFNPI["LAT_X"] = x_list
CinciPUFNPI["LON_y"] = y_list
CinciPUFNPI_final = CinciPUFNPI[CinciPUFNPI.CENSUS_BLOCK_ID != -1]
# Save to file
print (CinciPUFNPI_final.shape)
CinciPUFNPI_final.to_csv('./CinciPUFNPI_processed.csv', index=False)

ValueError: Length of values does not match length of index

In [ ]:
start = time.time()
block_list = []
x_list = []
y_list = []
idx = 0
for address in CinciBupTrtmntFac.Address:
    [census_block, coords_dict] = address_to_censusblock(address)
    block_list.append(census_block)
    x_list.append(coords_dict["x"])
    y_list.append(coords_dict["y"])
    idx += 1
    if idx%500 == 0:
        print (idx, "coordinates converted")
    if census_block == -1:
        print (address)
print ("total time:",(time.time() - start)/60., "mins")

In [ ]:
tract_list = [str(block)[:11] for block in block_list]
CinciBupTrtmntFac["CENSUS_BLOCK_ID"] = block_list
CinciBupTrtmntFac["CENSUS_TRACT_ID"] = tract_list
CinciBupTrtmntFac["LAT_X"] = x_list
CinciBupTrtmntFac["LON_y"] = y_list
CinciBupTrtmntFac_final = CinciPUFNPI[CinciPUFNPI.CENSUS_BLOCK_ID != -1]
# Save to file
print (CinciBupTrtmntFac_final.shape)
CinciBupTrtmntFac_final.to_csv('./CinciBupTrtmntFac_processed.csv', index=False)

In [ ]:
start = time.time()
block_list = []
x_list = []
y_list = []
idx = 0
for address in CincinnatiBuprPhysLoc.Address:
    [census_block, coords_dict] = address_to_censusblock(address)
    block_list.append(census_block)
    x_list.append(coords_dict["x"])
    y_list.append(coords_dict["y"])
    idx += 1
    if idx%500 == 0:
        print (idx, "coordinates converted")
    if census_block == -1:
        print (address)
print ("total time:",(time.time() - start)/60., "mins")

In [ ]:
tract_list = [str(block)[:11] for block in block_list]
CincinnatiBuprPhysLoc["CENSUS_BLOCK_ID"] = block_list
CincinnatiBuprPhysLoc["CENSUS_TRACT_ID"] = tract_list
CincinnatiBuprPhysLoc["LAT_X"] = x_list
CincinnatiBuprPhysLoc["LON_y"] = y_list
CincinnatiBuprPhysLoc_final = CincinnatiBuprPhysLoc[CincinnatiBuprPhysLoc.CENSUS_BLOCK_ID != -1]
# Save to file
print (CincinnatiBuprPhysLoc_final.shape)
CincinnatiBuprPhysLoc_final.to_csv('./CincinnatiBuprPhysLoc_processed.csv', index=False)